# Graph basics

docs: [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview)

Every graph consists of nodes and edges.

A typical node might represent an agent with a specific task, or some pre-/post-processing step that should run before/after the agent.

## Nodes

- A node has a name and an `.invoke()` method (a function that runs when the node is called)
- `.invoke()` should take the parameter `state: MessageState` (this is where the message history is stored)
- `.invoke()` should return a dict containing a list of new messages, e.g.: `{"messages": ["some new message"]}`

## Edges

- Edges are used to connect nodes
- Regular edges have a `start_key` (the name of the current node) and an `end_key` (the name of the next node)
- Each key can either be a command (like `__start__`, `__end__`) or the name of a node (e.g., `NodeA`, `NodeB`)
- The very first edge should have `start_key="__start__"`, marking the start of the graph
- The very last edge should have `end_key="__end__"`, marking the end of the graph

## Edge examples

| start_key   | end_key   | result                                  |
| ----------- | --------- | --------------------------------------- |
| `__start__` | `NodeA`   | Go from the start of the graph to NodeA |
| `NodeA`     | `NodeB`   | Go from NodeA to NodeB                  |
| `NodeB`     | `__end__` | Go from NodeB to the end of the graph   |


In [ ]:
from langchain.messages import HumanMessage
from langgraph.graph import MessagesState, StateGraph


In [ ]:
class Node:
    def __init__(self, name: str):
        self.name = name

    def invoke(self, state: MessagesState):
        message = f"This is a response from {self.name}"
        return {"messages": [message]}

# Initiate the nodes
node_a = Node(name="NodeA")
node_b = Node(name="NodeB")

# Add the nodes to the graph
graph_builder = StateGraph(MessagesState)
graph_builder.add_node(node=node_a.name, action=node_a.invoke)
graph_builder.add_node(node=node_b.name, action=node_b.invoke)

# Connect the nodes
graph_builder.add_edge(start_key="__start__", end_key=node_a.name)
graph_builder.add_edge(start_key=node_a.name, end_key=node_b.name)
graph_builder.add_edge(start_key=node_b.name, end_key="__end__")

# Compile and display graph
graph = graph_builder.compile()
display(graph)


In [ ]:
# Get and print output
messages = [HumanMessage("Some user input")]
output = graph.invoke({"messages": messages})
for i, msg in enumerate(output["messages"]):
    print(f"{i + 1}: {msg.content}")
